# Phase 2: Data Modeling & KPI Creation

In [3]:
# --------------------------------------------
# Phase 2: Data Modeling & KPI Design
# --------------------------------------------

import pandas as pd

DATA_PATH = "../Sampled"

# Load sampled data
purchases = pd.read_csv(f"{DATA_PATH}/purchases_sampled.csv")
sales = pd.read_csv(f"{DATA_PATH}/sales_sampled.csv")
begin_inv = pd.read_csv(f"{DATA_PATH}/begin_inventory_sampled.csv")
end_inv = pd.read_csv(f"{DATA_PATH}/end_inventory_sampled.csv")
invoices = pd.read_csv(f"{DATA_PATH}/vendor_invoice_sampled.csv")

# Vendor Procurement Model

In [17]:
vendor_procurement = (
    purchases
    .groupby(["VendorNumber", "VendorName"], as_index=False)
    .agg(
        total_purchase_qty=("Quantity", "sum"),
        total_purchase_cost=("Dollars", "sum"),
        avg_unit_cost=("PurchasePrice", "mean")
    )
)
vendor_procurement.head()

,VendorNumber,VendorName,total_purchase_qty,total_purchase_cost,avg_unit_cost
0,54,AAPER ALCOHOL & CHEMICAL CO,1,105.07,105.070000
1,60,ADAMBA IMPORTS INTL INC,708,11461.20,16.218065
2,105,ALTAMAR BRANDS LLC,72,2496.36,34.751538
3,200,AMERICAN SPIRITS EXCHANGE,36,350.28,9.730000
4,287,APPOLO VINEYARDS LLC,48,501.72,10.452500


# Freight Cost Integration

In [23]:
freight = (
    invoices
    .groupby(["VendorNumber", "VendorName"], as_index=False)
    .agg(total_freight=("Freight", "sum"))
)

vendor_procurement = vendor_procurement.merge(
    freight, on=["VendorNumber", "VendorName"], how="left"
)

vendor_procurement["freight_ratio"] = (
    vendor_procurement["total_freight"] /
    vendor_procurement["total_purchase_cost"]
)
vendor_procurement.describe()

,VendorNumber,total_purchase_qty,total_purchase_cost,avg_unit_cost,total_freight_x,freight_ratio,total_freight_y,total_freight
count,122.000000,122.000000,1.220000e+02,122.000000,120.000000,120.000000,120.000000,120.000000
mean,21458.245902,41303.385246,3.966762e+05,15.349365,3415.507000,0.009144,3415.507000,3415.507000
std,36327.133142,104386.668281,9.995466e+05,15.202601,8770.508641,0.006817,8770.508641,8770.508641
min,54.000000,1.000000,9.835000e+01,5.628571,0.630000,0.001506,0.630000,0.630000
25%,2556.500000,374.750000,5.214075e+03,8.856563,32.555000,0.006355,32.555000,32.555000
50%,6991.500000,1956.000000,2.349977e+04,12.346028,227.710000,0.008000,227.710000,227.710000
75%,10578.000000,29283.750000,2.687494e+05,16.113802,2388.650000,0.010282,2388.650000,2388.650000
max,173357.000000,814869.000000,7.610876e+06,135.586250,67337.640000,0.064569,67337.640000,67337.640000


# Vendor Sales Model

In [25]:
vendor_sales = (
    sales
    .groupby(["VendorNo", "VendorName"], as_index=False)
    .agg(
        total_sales_qty=("SalesQuantity", "sum"),
        total_sales_revenue=("SalesDollars", "sum"),
        avg_sales_price=("SalesPrice", "mean")
    )
    .rename(columns={"VendorNo": "VendorNumber"})
)
vendor_sales.head()

,VendorNumber,VendorName,total_sales_qty,total_sales_revenue,avg_sales_price
0,2,"IRA GOLDMAN AND WILLIAMS, LLP",11,370.89,30.990000
1,60,ADAMBA IMPORTS INTL INC,416,7071.84,16.997937
2,105,ALTAMAR BRANDS LLC,31,1429.69,49.190000
3,200,AMERICAN SPIRITS EXCHANGE,2,28.98,14.490000
4,287,APPOLO VINEYARDS LLC,9,139.41,15.490000


# Final Vendor Performance Table (MASTER TABLE)

In [29]:
vendor_performance = vendor_procurement.merge(
    vendor_sales, on="VendorNumber", how="left"
)

vendor_performance.fillna(0, inplace=True)
vendor_performance.head()

,VendorNumber,VendorName_x,total_purchase_qty,total_purchase_cost,avg_unit_cost,total_freight_x,freight_ratio,total_freight_y,total_freight,VendorName_y,total_sales_qty,total_sales_revenue,avg_sales_price
0,54,AAPER ALCOHOL & CHEMICAL CO,1,105.07,105.070000,0.00,0.000000,0.00,0.00,0,0.0,0.00,0.000000
1,60,ADAMBA IMPORTS INTL INC,708,11461.20,16.218065,54.26,0.004734,54.26,54.26,ADAMBA IMPORTS INTL INC,416.0,7071.84,16.997937
2,105,ALTAMAR BRANDS LLC,72,2496.36,34.751538,17.56,0.007034,17.56,17.56,ALTAMAR BRANDS LLC,31.0,1429.69,49.190000
3,200,AMERICAN SPIRITS EXCHANGE,36,350.28,9.730000,0.63,0.001799,0.63,0.63,AMERICAN SPIRITS EXCHANGE,2.0,28.98,14.490000
4,287,APPOLO VINEYARDS LLC,48,501.72,10.452500,2.04,0.004066,2.04,2.04,APPOLO VINEYARDS LLC,9.0,139.41,15.490000


# Core Business KPIs

In [37]:
vendor_performance["gross_profit"] = (
    vendor_performance["total_sales_revenue"] -
    vendor_performance["total_purchase_cost"]
)

vendor_performance["sales_to_cost_ratio"] = (
    vendor_performance["total_sales_revenue"] /
    vendor_performance["total_purchase_cost"]
)

vendor_performance["gross_margin_pct"] = (
    vendor_performance["gross_profit"] /
    vendor_performance["total_sales_revenue"]
) * 100


# Save Final Modeled Output

In [15]:
vendor_performance.to_csv(
    "final_vendor_performance.csv",
    index=False
)